## Model extraction and reproduction

### Extract parameters from a .pt file

In [125]:
import os
import torch
import random
import numpy as np
def seed_everything(seed=20):
    """set seed for all"""
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [126]:
# load the model
import torch
from torch import nn
sent_model = torch.load('A_model_1.pt', map_location='cpu')
sent_model.eval()
for param in sent_model.parameters():
    param.requires_grad = False
print(sent_model)

Sequential(
  (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=768, out_features=128, bias=True)
  (2): GELU()
  (3): Linear(in_features=128, out_features=3, bias=True)
)


In [127]:
# get model structure
sent_model_structure = '|'.join([layer_str.split('): ')[1] for layer_str in str(sent_model).split('\n')[1:-1]])
print('\n'.join(sent_model_structure.split('|')))

BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Linear(in_features=768, out_features=128, bias=True)
GELU()
Linear(in_features=128, out_features=3, bias=True)


In [128]:
# get model parameters
with torch.no_grad():
    sent_model_parameters = {name:val.detach() for name, val in sent_model.named_parameters()}

In [129]:
# Since the model uses normalization, we also need to send the mean and variance
sent_model_parameters['0.running_mean'] = sent_model[0].running_mean
sent_model_parameters['0.running_var'] = sent_model[0].running_var

In [130]:
# print weight name and shape
sent_model_parameters_names = list(sent_model_parameters.keys())
sent_model_parameters_vals = list(sent_model_parameters.values())
for name, val in zip(sent_model_parameters_names, sent_model_parameters_vals):
    print(f'{name:8s}\t{val.__class__.__name__}\t{list(val.shape)}'.expandtabs(2))
sent_model_parameters_names = '|'.join(sent_model_parameters_names)

0.weight  Tensor  [768]
0.bias    Tensor  [768]
1.weight  Tensor  [128, 768]
1.bias    Tensor  [128]
3.weight  Tensor  [3, 128]
3.bias    Tensor  [3]
0.running_mean  Tensor  [768]
0.running_var Tensor  [768]


In [131]:
# We will send 3 parameters to a client
'''
    sent_model_structure
    sent_model_parameters_names
    sent_model_parameters_vals
''';

### Test Reproductivity

In [132]:
# Stimulate a client who the receives the parameters
received_model_structure = sent_model_structure
received_model_parameters_names = [f'[{name.split(".")[0]}].{name.split(".")[1]}'
                                   for name in sent_model_parameters_names.split('|')]
received_model_parameters_vals = sent_model_parameters_vals

In [133]:
# initialize the model
received_model = nn.Sequential(
    *[eval('nn.' + layer) for layer in received_model_structure.split('|')]
)
# print part of the initial model weight
print(received_model[0].weight.detach()[:10])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [134]:
# assign weights to each layer
for parameters_name, parameters_val in zip(received_model_parameters_names, received_model_parameters_vals):
    exec(f'received_model{parameters_name} = nn.Parameter(parameters_val)')

In [135]:
# make sure the weight has been changed
print(received_model[0].weight.detach()[:10])

tensor([0.9913, 1.0620, 1.0096, 0.9389, 1.0044, 1.0548, 0.9228, 0.9387, 1.0055,
        0.9827])


### Make sure two models have the same output for a given input

In [136]:
# Random input for the model
random_input = torch.randn(32, 768)
sent_model_output = sent_model(random_input)
sent_model_output[:5]

tensor([[ 0.6840, -0.6742, -0.8874],
        [ 0.4475, -0.3338, -1.1385],
        [ 0.6013, -0.7323, -0.7802],
        [ 0.4355, -0.3431, -0.8839],
        [ 0.5737, -0.6191, -0.7754]])

In [137]:
# test the output of sent_model
sent_model.eval()
for param in sent_model.parameters():
    param.requires_grad = False
sent_model_output = sent_model(random_input)
sent_model_output[:5]

tensor([[ 0.6840, -0.6742, -0.8874],
        [ 0.4475, -0.3338, -1.1385],
        [ 0.6013, -0.7323, -0.7802],
        [ 0.4355, -0.3431, -0.8839],
        [ 0.5737, -0.6191, -0.7754]])

In [138]:
# test the output of received_model
received_model.eval()
for param in received_model.parameters():
    param.requires_grad = False
received_model_output = received_model(random_input)
received_model_output[:5]

tensor([[ 0.6840, -0.6742, -0.8874],
        [ 0.4475, -0.3338, -1.1385],
        [ 0.6013, -0.7323, -0.7802],
        [ 0.4355, -0.3431, -0.8839],
        [ 0.5737, -0.6191, -0.7754]])

In [139]:
if (sent_model_output == received_model_output).all():
    print('Success! Two models have the same output')
else:
    print('Fail! Please check if any of the parameters are different')

Success! Two models have the same output
